In [1]:
import numpy as np
import paddle_quantum as pq
from paddle_quantum.ansatz.circuit import Circuit

from paddle_quantum.linalg import haar_unitary
from paddle_quantum.qinfo import state_fidelity
from paddle_quantum.state.common import to_state
from scipy import integrate

from scipy.stats import entropy

num_qubit = 4
num_sample = 2000

C:\Users\luccas.marim\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openfermion\hamiltonians\hartree_fock.py:11: DeprecationWarning: Please use `OptimizeResult` from the `scipy.optimize` namespace, the `scipy.optimize.optimize` namespace is deprecated.
  from scipy.optimize.optimize import OptimizeResult
C:\Users\luccas.marim\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\paddle\tensor\creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:


In [2]:
def p_F_haar(n, s, b=50):
    
    f_list = list()

    for i in range(s):
        u1 = haar_unitary(n)
        phi1 = u1[:,0]
        rho1 = to_state(np.outer(phi1, phi1.conj()))
        u2 = haar_unitary(n)
        phi2 = u2[:,0]
        rho2 = to_state(np.outer(phi2, phi2.conj()))
        f_list.append(state_fidelity(rho1, rho2)**2)

    f_list = np.array(f_list)
    sample_distribution, _ = np.histogram(f_list, bins=np.linspace(0, 1, num=b), density=True)
    theory_distribution = np.zeros_like(sample_distribution)

    for index in range(len(theory_distribution)):
        def p_continues(f):
            return (2 ** n - 1) * (1 - f) ** (2 ** n - 2)
        lower = 1/b*index
        upper = lower + 1/b
        theory_distribution[index], _ = integrate.quad(p_continues,lower,upper)

    return sample_distribution, theory_distribution

In [3]:
pq.set_backend('density_matrix')

def p_F_cel(n, d, s, b=50):

    pq.set_backend('state_vector')
    f_list = list()
    for _ in range(int(s / 2)):

        # Aqui colocamos o circuito quântico variacional 2 vezes

        cir1 = Circuit(num_qubit)
        for i in range(num_qubit):
            cir1.rx(i)
            cir1.rz(i)
        cir1.crz([3,2])
        cir1.crz([2,1])
        cir1.crz([1,0])

        cir2 = Circuit(num_qubit)
        for i in range(num_qubit):
            cir2.rx(i)
            cir2.rz(i)
        cir2.crz([3,2])
        cir2.crz([2,1])
        cir2.crz([1,0])



        rho1 = cir1(pq.state.zero_state(n))
        rho2 = cir2(pq.state.zero_state(n))
        
        f_list.append(abs(np.inner(rho1.data.numpy(), rho2.data.numpy().conj()))**2)
        
    f_list = np.array(f_list)

    distribution, _ = np.histogram(f_list, bins=np.linspace(0, 1, num=b), density=True)

    return distribution

In [4]:
p_haar_4qubit, theory_haar_4qubit = p_F_haar(num_qubit, num_sample)

p_cel = p_F_cel(num_qubit, 1, num_sample)
expr = entropy(p_cel, theory_haar_4qubit)

Expr_cel = list()
Expr_cel.append(expr)

print("The expressibility is", np.around(Expr_cel, decimals=4))

C:\Users\luccas.marim\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\paddle\fluid\framework.py:1104: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif dtype == np.bool:


The expressibility is [0.2892]
